Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)
# import requests
# import datetime

In [10]:
# Import data from file
df_ctybks = pd.read_csv('data/df_citybikes.csv')
df_fsq = pd.read_csv('data/df_fsq_all_loci102.csv')
# df_yelp = pd.read_csv('data/df_yelp_all_loci102.csv')

In [18]:
print(df_ctybks.shape)
print(df_fsq.shape)

(241, 12)
(2410, 15)


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [12]:
# Create a new column with the coordinate data we need, reformated to str dtype.

# Grab just the two columns we need lat and lon
df_ctybks.dropna(subset=['latitude', 'longitude'], inplace=True)

# Creation of a new variable with lon and lat together, label the column header 'lat-long'
df_ctybks['lat-long'] = df_ctybks['latitude'].astype(str) + ',' + df_ctybks['longitude'].astype(str)
df_ctybks = df_ctybks[df_ctybks['lat-long'] != '0.0,0.0']
df_ctybks.head()

,Unnamed: 0,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID,lat-long
0,0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,1,"49.262487,-123.114397"
1,1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,4,"49.274566,-123.121817"
2,2,0005 Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,5,"49.279764,-123.110154"
3,3,0007 12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,7,"49.260599,-123.113504"
4,4,0008 8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,8,"49.264215,-123.117772"


In [17]:
df_merged = pd.merge(df_ctybks, df_fsq, how='inner', on='lat-long')
df_merged.shape

(2410, 26)

In [ ]:
# Checking the types of data
df = df_merged.copy()
df.dtypes

In [ ]:
# WIP -----------------------------------------

In [ ]:
# Dropping irrelevant columns
# df = df.drop(['col_a', 'col_b', 'col_c_etc.'], axis=1)

# Recheck headings:
# df.head(5)

In [ ]:
# Renaming the columns if needed - say if you wanted them "short and pithy Dr B!"
# df = df.rename(columns = { 'Lengthy_current':'LP', 'doggy_booty_toot':'Lucy_etc.' })
# df.head(5)

In [ ]:
# Checking for the presence and location of duplicate rows
# duplicate_rows_df = df[df.duplicated()]

# And compare the difference in the DF shape:
# print(df.shape)
# print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
# Assess the location and content of the duplications, are they appropriate?
# If not, we can drop them:

# First a count of the total rows to compare to #of duplicate rows
# df.count()

# Then we drop, and recheck:
# df = df.drop_duplicates()
# df.head(5)

In [ ]:
# Drop any missing or null values:
# print(df.isnull().sum())

In [ ]:
# We'll drop the full rows that contain null values:
# df = dfdropna()
# df.count()

In [ ]:
# Validating the dropped missing values
# print(df.isnull().sum())

In [ ]:
# Detecting outliers on numerical values
# sns.boxplot(x=df['col_with_numerical_value'])

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.